In [7]:
import urllib.request, json, datetime, time

#documentation of API: 
#https://www.transitchicago.com/assets/1/6/cta_Customer_Alerts_API_Developer_Guide_and_Documentation_20160929.pdf

alertroutestring = {
    "location1":"Red,147,22,49,Brn,Blue,Org",
    }
#defines alerts to search for, note short names of L lines from API docs

def CTAalerts(locstring):
    if alertroutestring[locstring] == "":
        return []
    alerturl = "http://www.transitchicago.com/api/1.0/alerts.aspx?routeid="+alertroutestring[locstring]+"&outputType=JSON&accessibility=FALSE&activeonly=TRUE&recentdays=7"
    #by changing parameters in that URL the extent of alerts outputted can be altered
    #this includes only currently active alerts, excludes long-term alerts, and excludes accessibility alerts
    
    with urllib.request.urlopen(alerturl) as url:
        alerts = json.loads(url.read().decode())

    if alerts["CTAAlerts"]["ErrorCode"] != "0":
        return ""
    alertstoprint = []
    for disruption in alerts["CTAAlerts"]["Alert"]:
        line = disruption["ShortDescription"]
        line = line.replace("\r\n"," ")
        headline = disruption["Headline"]
        headline = headline.replace("Lake Shore Drive","LSD")
        headline = headline.replace("Express", "Exp")
        headline = headline.replace("Exp.", "Exp")
        #the above shortens the headline to keep it short.
        #Unfortunately the CTA is very inconsistent about naming alerts.
        #Often the name of the service affected is included, but not always.
        
        description = alerttextfixer(disruption["FullDescription"]["#cdata-section"])
        # try:
        #     backcolor = disruption["ImpactedService"]["Service"]["ServiceBackColor"]
        #     textcolor = disruption["ImpactedService"]["Service"]["ServiceTextColor"]
        # except:
        backcolor = "059"
        textcolor = "ffffff"
        link = disruption["AlertURL"]["#cdata-section"]
        alertstoprint.append([headline,backcolor,textcolor,link,description])
    return alertstoprint

def alerttextfixer(desc):
    desc  = desc.replace("\r\n"," ")
    desc = desc.replace("\n","")
    desc = desc.replace("  "," ")
    desc = desc.replace("&nbsp;"," ")
    desc = desc.replace("&#39;","\'")
    while "<" in desc and ">" in desc:
        loc1 = desc.find("<")
        loc2 = desc.find(">",loc1)+1
        desc = desc.replace(desc[loc1:loc2],"")
    return desc

In [8]:
CTAalerts("location1")

''